<a href="https://colab.research.google.com/github/SJZHZ/Multi-modal-Learning/blob/main/Copy_of_%E2%80%9Cassignment_2_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2
You will tackle with a sentiment classification task using LSTM model and attention mechanism in this assigment.

# Dependencies
Please make sure that you are using **GPU** to accelarate computation.

Colab FAQ: https://research.google.com/colaboratory/faq.html

## Import dependencies

In [ ]:
import torch
import os
import collections
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from tqdm import tqdm
import math
import random
import numpy as np

In [ ]:
# Set up your device 
cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if cuda else "cpu")

# The assertion is to make sure GPU is available
assert cuda == True

In [ ]:
# Set up random seed to 1008. Do not change the random seed.
# Yes, these are all necessary when you run experiments!
seed = 1008
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


## Data
The script below will download the required sentiment analysis data.

Data folder will be visible in the Colab file-explorer pane, which is loacted at left side of the page.


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jqYJ9jhjukhXvEk4GnMAPYE-SvhSG24i' -O data.zip
!unzip data.zip

--2022-11-14 18:54:09--  https://docs.google.com/uc?export=download&id=1jqYJ9jhjukhXvEk4GnMAPYE-SvhSG24i
Resolving docs.google.com (docs.google.com)... 172.217.194.101, 172.217.194.100, 172.217.194.113, ...
Connecting to docs.google.com (docs.google.com)|172.217.194.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-4g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fttq9uu5fkudr8i3a8cg58af84dqe56d/1668452025000/14455349759236007028/*/1jqYJ9jhjukhXvEk4GnMAPYE-SvhSG24i?e=download&uuid=8ed91f20-9d4a-4122-899c-4d80fe9ebf39 [following]
--2022-11-14 18:54:13--  https://doc-10-4g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/fttq9uu5fkudr8i3a8cg58af84dqe56d/1668452025000/14455349759236007028/*/1jqYJ9jhjukhXvEk4GnMAPYE-SvhSG24i?e=download&uuid=8ed91f20-9d4a-4122-899c-4d80fe9ebf39
Resolving doc-10-4g-docs.googleusercontent.com (doc-10-4g-docs.googleusercontent.com)... 74.125.24.132, 2404:68

## Corpus
Glove will be used as the word embedding tool in this assigment.

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2022-11-14 18:54:21--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-14 18:54:21--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-14 18:54:22--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

# Preprocess
Preprocess data, then construct dataloader and vocabulary.

## Load Glove pretrained word embedding.

In [ ]:
# TODO
def LoadGloveModel(gloveFile, dim):    # 从文件中加载Glove预训练模型。返回一个词典和一个矩阵
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    array = []
    model[0] = 0
    array.append([0] * dim)
    i = 1
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = i       # 序号
        array.append(embedding)    # 嵌入   
        i += 1
    array = np.array(array)
    print("Done.",len(model)," words loaded!")
    return model, array

dim = 300
GloveDict, GloveEmbedding = LoadGloveModel("glove.6B.300d.txt", 300)       # 100维
print(GloveDict['my'])
print(GloveEmbedding[0])

Loading Glove Model
Done. 400001  words loaded!
193
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


## Construct your own vocabulary without other corpus.
Hint: You should construct a vocabulary to map the word to index.

In [ ]:
# TODO
MyWord = collections.Counter()
f = open('sentiment'+'/train_text.txt')
for line in f:
    splitLine = line.split()
    MyWord.update(splitLine)    # 更新词频列表
print(len(MyWord))
MyDict = {}
i = 0
MyDict[0] = 0
for word in MyWord:
  if MyWord[word] > 0:      # 超过1次
    i += 1
    MyDict[word] = i
print(len(MyDict))
Embedding300 = nn.Embedding(len(MyDict), 300)
templist = []
for i in range(len(MyDict)):
  b = Embedding300(torch.LongTensor([i])).detach().numpy()[0]
  templist.append(b)
MyEmbedding = np.array(templist)
print(MyEmbedding.shape)

106360
106361
(106361, 300)


## Load data
Load data and construct dataloader.

In [ ]:
data_dir = 'sentiment'

# TODO
def LoadText(DataFile):    # 从文件中加载训练文本（每行是一句话）。返回一个列表
    print("Loading Text")
    f = open(DataFile,'r')
    temp_data = []
    for line in f:
        splitLine = line.split()
        temp_data.append(splitLine)
    print("Done.",len(temp_data)," sentences loaded!")
    return temp_data


maxlen = 40    # 句子对齐到最大长度
def LoadData(Text, Dict, dim):
  print("Loading Data")
  temp_data = []
  for line in Text:
    temp_sentence = []
    for word in line:
      if word in Dict:      # 匹配的才算
        temp_sentence.append(Dict[word])
    temp_len = len(temp_sentence)
    for i in range(maxlen - temp_len): 
      temp_sentence.append(Dict[0])
    temp_data.append(temp_sentence)
  print("Done.",len(temp_data)," lines loaded!")
  return temp_data

def LoadLabel(LabelFile):     # 从文件中加载训练标签（每行一个数值）。返回一个列表
    print("Loading Label")
    f = open(LabelFile,'r')
    temp_labels = []
    for line in f:
        splitLine = line.split()
        label = int(splitLine[-1])  # 每行最后一个位,读入的是字符串
        label_vec = [0] * 3   # BiRNN模型输出3个类别,需要转换为n维向量
        label_vec[label] = 1
        temp_labels.append(label_vec)
    print("Done.",len(temp_labels)," lines loaded!")
    return temp_labels

def MakeDataset(textname, labelname, dictname, dim):
  temp_text = LoadText(textname)
  temp_data = LoadData(temp_text, dictname, dim)
  temp_label = LoadLabel(labelname)
  temp = torch.from_numpy(np.array(temp_label))
  print(temp.shape, temp.dtype)
  dataset = TensorDataset(torch.from_numpy(np.array(temp_data)),torch.from_numpy(np.array(temp_label)))
  return dataset

# 根据想用的词典
tempdict, tempdim = GloveDict, 300
# tempdict, tempdim = MyDict, 300

val_Dataset = MakeDataset(data_dir + '/val_text.txt', data_dir + '/val_labels.txt', tempdict, tempdim)
test_Dataset = MakeDataset(data_dir + '/test_text.txt', data_dir + '/test_labels.txt', tempdict, tempdim)
train_Dataset = MakeDataset(data_dir + '/train_text.txt', data_dir + '/train_labels.txt', tempdict, tempdim)

batch_size = 128
val_Loader = DataLoader(val_Dataset, shuffle=True, batch_size=batch_size)
test_Loader = DataLoader(test_Dataset, shuffle=True, batch_size=batch_size)
train_Loader = DataLoader(train_Dataset, shuffle=True, batch_size=batch_size)

Loading Text
Done. 2000  sentences loaded!
Loading Data
Done. 2000  lines loaded!
Loading Label
Done. 2000  lines loaded!
torch.Size([2000, 3]) torch.int64
Loading Text
Done. 12284  sentences loaded!
Loading Data
Done. 12284  lines loaded!
Loading Label
Done. 12284  lines loaded!
torch.Size([12284, 3]) torch.int64
Loading Text
Done. 45615  sentences loaded!
Loading Data
Done. 45615  lines loaded!
Loading Label
Done. 45615  lines loaded!
torch.Size([45615, 3]) torch.int64


# Model
Bidirectional LSTM and attention mechanism will be used in this section.

## Model Zoo

In [ ]:
class BiRNN(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, pretrained_embedding=None, **kwargs):
        super(BiRNN, self).__init__()
        if pretrained_embedding is None:
            self.embedding = nn.Embedding(vocab_size, embed_size)
        else:
            self.embedding= nn.Embedding.from_pretrained(torch.tensor(pretrained_embedding, dtype=torch.float), freeze=True)
        self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.decoder = nn.Sequential(nn.Linear(4 * num_hiddens, num_hiddens), nn.Linear(num_hiddens, 3))

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        self.encoder.flatten_parameters()
        outputs, _ = self.encoder(embeddings)
        encoding = torch.cat((outputs[:,0,:], outputs[:,-1,:]), dim=1)
        outs = self.decoder(encoding)
        return outs

In [ ]:
class BiRNN_attention(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, pretrained_embedding=None, **kwargs):
        super(BiRNN_attention, self).__init__()
        if pretrained_embedding is None:
            self.embedding = nn.Embedding(vocab_size, embed_size)
        else:
            self.embedding = nn.Embedding.from_pretrained(torch.tensor(pretrained_embedding, dtype=torch.float),
                                                          freeze=True)
        self.encoder = nn.LSTM(embed_size, num_hiddens, num_layers=num_layers, bidirectional=True, batch_first=True)
        self.weight_W = nn.Parameter(torch.Tensor(2 * num_hiddens, 2 * num_hiddens))
        self.weight_proj = nn.Parameter(torch.Tensor(2 * num_hiddens, 1))

        self.decoder = nn.Sequential(nn.Linear(2 * num_hiddens, num_hiddens), nn.Linear(num_hiddens, 3))
        nn.init.uniform_(self.weight_W, -0.1, 0.1)
        nn.init.uniform_(self.weight_proj, -0.1, 0.1)


    def forward(self, inputs):
        mask = 1 - torch.clamp(inputs, min=0, max=1)
        embeddings = self.embedding(inputs)
        states, hidden = self.encoder(embeddings.permute([0, 1, 2]))
        u = torch.tanh(torch.matmul(states, self.weight_W))
        att = torch.matmul(u, self.weight_proj)
        att = att + mask.unsqueeze(2) * -1e7
        att_score = F.softmax(att, dim=1)
        scored_x = states * att_score
        encoding = torch.sum(scored_x, dim=1)
        outputs = self.decoder(encoding)

        return outputs

## Training
You should train two models above with Glove pretrained word embedding and random initialized word embedding.

Evaluation on the validation set and print out accuracy after training one epoch is required.

You can tune some parameters and try different techniques, such as learning rate scheduler.

In [ ]:
def train(model, Loader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for sentence, label in Loader:
        optimizer.zero_grad()
        sentence = sentence.to(device)
        label = label.to(device)

        pred = model(sentence).squeeze(1)
        loss = criterion(pred, label.float()) # label本来是整数列表
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(Loader)

def test(model, Loader, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    for sentence, label in Loader:
        sentence = sentence.to(device)
        label = label.to(device)
        pred = model(sentence).squeeze(1)
        loss = criterion(pred, label.float())
        epoch_loss += loss.item()
        epoch_acc += (pred.argmax(dim=1) == label.argmax(dim=1)).float().mean().cpu().item()
        
    return epoch_loss / len(Loader), epoch_acc / len(Loader)


In [ ]:
# Train BiRNN with Glove pretrained word embedding
# TODO
vocabsize, embedsize = GloveEmbedding.shape[0], GloveEmbedding.shape[1]

Model1 = BiRNN(vocab_size=vocabsize, embed_size=embedsize, num_hiddens=20, num_layers=2, pretrained_embedding=GloveEmbedding).to(device)
criterion = nn.CrossEntropyLoss()
# criterion = nn.BCELoss()
optimizer = torch.optim.Adam(Model1.parameters())

for iter in range(5):    # 写的很烂,次数太多反而准确率变低
  train_loss = train(model=Model1, Loader=train_Loader, optimizer=optimizer, criterion=criterion)
  test_loss, test_acc = test(model=Model1, Loader=val_Loader, criterion=criterion)
  print(iter, ": train ", train_loss, ", val ", test_loss, test_acc)
test_loss, test_acc = test(model=Model1, Loader=test_Loader, criterion=criterion)
print("test loss", test_loss, "test acc", test_acc)

0 : train  0.9184199730221297 , val  0.8688862584531307 0.5880859382450581
1 : train  0.8289449710805877 , val  0.8563771359622478 0.5982421897351742
2 : train  0.8048629949406749 , val  0.8530773296952248 0.5984374992549419
3 : train  0.7852504493809548 , val  0.8422710783779621 0.6081054694950581
4 : train  0.7694343532166895 , val  0.8495704121887684 0.6119140647351742
test loss 0.8815583176910877 test acc 0.6035576276481152


In [ ]:
# Train BiRNN without pretrained word embedding
# TODO
vocabsize, embedsize = MyEmbedding.shape[0], MyEmbedding.shape[1]

Model2 = BiRNN(vocab_size=vocabsize , embed_size=embedsize, num_hiddens=200, num_layers=2, pretrained_embedding=MyEmbedding).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Model2.parameters())

for iter in range(3):
  train_loss = train(model=Model2, Loader=train_Loader, optimizer=optimizer, criterion=criterion)
  test_loss, test_acc = test(model=Model2, Loader=val_Loader, criterion=criterion)
  print(iter, ": train ", train_loss, ", val ", test_loss, test_acc)
test_loss, test_acc = test(model=Model2, Loader=test_Loader, criterion=criterion)
print("test loss", test_loss, "test acc", test_acc)

0 : train  0.930183180240022 , val  0.8794585727155209 0.5646484382450581
1 : train  0.7869017695178505 , val  0.8441049233078957 0.6104492209851742
2 : train  0.6356490883840566 , val  0.8743134699761868 0.6092773452401161
test loss 1.0801431400080521 test acc 0.5492612775415182


In [ ]:
# Train BiRNN_attention with Glove pretrained embedding
# TODO
vocabsize, embedsize = GloveEmbedding.shape[0], GloveEmbedding.shape[1]

Model3 = BiRNN_attention(vocab_size=vocabsize , embed_size=embedsize, num_hiddens=20, num_layers=2, pretrained_embedding=GloveEmbedding).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Model3.parameters())

for iter in range(5):
  train_loss = train(model=Model3, Loader=train_Loader, optimizer=optimizer, criterion=criterion)
  test_loss, test_acc = test(model=Model3, Loader=val_Loader, criterion=criterion)
  print(iter, ": train ", train_loss, ", val ", test_loss, test_acc)
test_loss, test_acc = test(model=Model3, Loader=test_Loader, criterion=criterion)
print("test loss", test_loss, "test acc", test_acc)

0 : train  0.9117326836626068 , val  0.8522169962525368 0.58984375
1 : train  0.8178155744443086 , val  0.830701969563961 0.6117187514901161
2 : train  0.7954208246466159 , val  0.8367760479450226 0.6081054694950581
3 : train  0.7792759522670457 , val  0.8300911784172058 0.6201171875
4 : train  0.7627293567697541 , val  0.8295808471739292 0.6142578125
test loss 0.9039642363786697 test acc 0.6000582786897818


In [ ]:
# Train BiRNN_attention without pretrained word embedding
# TODO
vocabsize, embedsize = MyEmbedding.shape[0], MyEmbedding.shape[1]

Model4 = BiRNN(vocab_size=vocabsize , embed_size=embedsize, num_hiddens=200, num_layers=2, pretrained_embedding=MyEmbedding).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Model4.parameters())

for iter in range(3):
  train_loss = train(model=Model4, Loader=train_Loader, optimizer=optimizer, criterion=criterion)
  test_loss, test_acc = test(model=Model4, Loader=val_Loader, criterion=criterion)
  print(iter, ": train ", train_loss, ", val ", test_loss, test_acc)
test_loss, test_acc = test(model=Model4, Loader=test_Loader, criterion=criterion)
print("test loss", test_loss, "test acc", test_acc)

0 : train  0.9329534366685135 , val  0.880100667476654 0.5702148452401161
1 : train  0.7887001314750907 , val  0.844881996512413 0.6101562492549419
2 : train  0.6334546577362787 , val  0.8882623575627804 0.6125000007450581
test loss 1.117300431554516 test acc 0.542000062763691


# Report (optional)
You can briefly report what strategies you attempted in this assignment.

1. 不太明白怎么自己作vocabulary，我是把test data的单词拿来建一个dict，然后做成embedding
2. 把单词用dict映成序号，然后使用embedding表达。对于生词统一记为0合适吗？
3. 训练写的不太好，随着训练次数增加，val的loss越来越大，应该是发生了过拟合
4. 晚上把Colab的免费时长用完了。。。只能开个新号了